<a href="https://colab.research.google.com/github/junieberry/NLP-withPyTorch/blob/main/03_Yelp/03_Yelp_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

ERROR: torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.


In [11]:
import torch
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

## 3.6 레스토랑 리뷰 감성 분류하기




### 3.6.1 옐트 리뷰 데이터셋

- Binary
- Light 버전 사용 (빠른 속도, 낮은 정확도)

- 훈련, 검증, 테스트로 분리

### 데이터 가져오기

In [17]:
!cd /content/drive/MyDrive/nlp-with-pytorch/chapter_3/data
!chmod 755 get-all-data.sh
!./get-all-data.sh

Trying to fetch /content/drive/MyDrive/nlp-with-pytorch/chapter_3/data/yelp/raw_train.csv
12536it [00:05, 2429.59it/s]
Trying to fetch /content/drive/MyDrive/nlp-with-pytorch/chapter_3/data/yelp/raw_test.csv
848it [00:01, 711.21it/s]
Trying to fetch /content/drive/MyDrive/nlp-with-pytorch/chapter_3/data/yelp/reviews_with_splits_lite.csv
1217it [00:00, 2726.43it/s]


### 데이터 비율 및 개수 맞추기

In [41]:
args = Namespace(
    raw_train_dataset_csv="yelp/raw_train.csv",
    raw_test_dataset_csv="yelp/raw_test.csv",
    proportion_subset_of_train=0.1,
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv="yelp/reviews_with_splits_lite.csv",
    seed=1337
)

In [19]:
train_reviews = pd.read_csv(args.raw_train_dataset_csv, header=None, names=['rating', 'review'])

In [32]:
train_reviews

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...
...,...,...
559995,2,Ryan was as good as everyone on yelp has claim...
559996,2,Professional \nFriendly\nOn time AND affordabl...
559997,1,Phone calls always go to voicemail and message...
559998,1,Looks like all of the good reviews have gone t...


In [33]:
train_reviews.rating.value_counts()

2    280000
1    280000
Name: rating, dtype: int64

In [30]:
## 리뷰 클래스 비율 동일하게 하기 위해서 rating 별로 review 들을 구분해둠
by_rating = collections.defaultdict(list)
for _,row in train_reviews.iterrows():
  by_rating[row.rating].append(row.to_dict())

review_subset = []

## 우리는 10%만 사용할거니까 args.proportion_subset_of_train을 사용해서 잘라줌
for _, item_list in sorted(by_rating.items()):
  n_total = len(item_list)
  n_subset = int(args.proportion_subset_of_train*n_total)
  review_subset.extend(item_list[:n_subset])

review_subset = pd.DataFrame(review_subset)

In [29]:
review_subset

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,1,I don't know what Dr. Goldberg was like before...
2,1,I'm writing this review to give you a heads up...
3,1,Wing sauce is like water. Pretty much a lot of...
4,1,Owning a driving range inside the city limits ...
...,...,...
55995,2,I am not really an arts and crafts kind of guy...
55996,2,i absolutely love michael's! i used to scrapbo...
55997,2,The fact that I can generally get whatever I w...
55998,2,I've been frequenting Michaels lately because ...


In [31]:
review_subset.rating.value_counts()

2    28000
1    28000
Name: rating, dtype: int64

In [27]:
set(review_subset.rating)

{1, 2}

### 훈련, 검증, 테스트 세트

In [34]:
## 이것도 리뷰 클래스 별로 구분하기 위해 rating 별로 review 구분
by_rating = collections.defaultdict(list)
for _,row in train_reviews.iterrows():
  by_rating[row.rating].append(row.to_dict())


final_list = []
## 랜덤 시드를 정해주면 다음에 똑같이 할 수 있음!
np.random.seed(args.seed)

for item, item_list in sorted(by_rating.items()):
  print(item)
  print(item_list)
  np.random.shuffle(item_list)

  n_total = len(item_list)
  n_train = int(args.train_proportion * n_total)
  n_val = int(args.val_proportion * n_total)
  n_test = int(args.test_proportion * n_total)

  ## 어떤 데이터 셋인지 정보 추가
  for item in item_list[:n_train]:
    item['split'] = 'train'
    
  for item in item_list[n_train:n_train+n_val]:
    item['split'] = 'val'
        
  for item in item_list[n_train+n_val:n_train+n_val+n_test]:
      item['split'] = 'test'

  final_list.extend(item_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [35]:
final_reviews = pd.DataFrame(final_list)
final_reviews.split.value_counts()

train    392000
test      84000
val       84000
Name: split, dtype: int64

### 데이터 전처리

구두점 기호 앞뒤에 공백

구두점 아닌 기호 제거

In [36]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text
    
final_reviews.review = final_reviews.review.apply(preprocess_text)

In [37]:
final_reviews['rating'] = final_reviews.rating.apply({1: 'negative', 2: 'positive'}.get)

In [39]:
final_reviews

,rating,review,split
0,negative,the entrance was the impressive thing about th...,train
1,negative,"i m a mclover , and i had no problem nwith the...",train
2,negative,"less than good here , not terrible , but i see...",train
3,negative,i don t know if i can ever bring myself to go ...,train
4,negative,food was ok good but the service was terrible ...,train
...,...,...,...
559995,positive,i was in japan for two months almost years ago...,test
559996,positive,quick service ! clean .,test
559997,positive,"we looked in at jet , which had a line that wa...",test
559998,positive,i m coming up on living here a month now and i...,test


In [42]:
final_reviews.to_csv(args.output_munged_csv, index=False)